In [1]:
import sys
sys.path.append('../..')

In [2]:
from probability import *
from utils import print_table

# Basic Probability

In [3]:
variables = ['X1', 'X2']

In [4]:
j = JointProbDist(variables)
j

P(['X1', 'X2'])

In [5]:
for x1 in range(1, 7):
    for x2 in range(1, 7):
        j[x1, x2] = 1/36

In [6]:
j[(1,1)]

0.027777777777777776

---

# Bayesian Networks

http://www.cs.man.ac.uk/~gbrown/bayes_nets/#

![](./images/bayesian_network_traffic.png)

In [7]:
T, F = True, False

traffic = BayesNet([
    ('Rain', '', 0.41),
    ('Traffic', '', 0.15),
    ('Motorway', '', 0.01),
    ('Late', 'Rain Traffic Motorway',
     {(T, T, T): 0.8, (T, T, F): 0.98, (T, F, T): 0.2, (T, F, F): 0.3, (F, T, T): 0.25, (F, T, F): 0.24, (F, F, T): 0.001, (F, F, F): 0.05}),
    ('BossCalls', 'Late', {T: 0.8, F: 0.1}),
])

In [8]:
traffic

BayesNet([('Rain', ''), ('Traffic', ''), ('Motorway', ''), ('Late', 'Rain Traffic Motorway'), ('BossCalls', 'Late')])

In [9]:
traffic.variable_node('Late').cpt

{(True, True, True): 0.8,
 (True, True, False): 0.98,
 (True, False, True): 0.2,
 (True, False, False): 0.3,
 (False, True, True): 0.25,
 (False, True, False): 0.24,
 (False, False, True): 0.001,
 (False, False, False): 0.05}

## Exact Inference in Bayesian Networks
### Inference by Enumeration

In [10]:
# You took the Motorway

#motorway = enumeration_ask('Motorway', {'Rain': False}, traffic)
#print(motorway[True])

for k,v in traffic.variable_node('Motorway').cpt.items():
    print(v)

0.01


In [11]:
# The boss does not call given that you are late

call = enumeration_ask('BossCalls', {'Late': True}, traffic)
call[True]

0.8

In [12]:
# You are late when its raining & there is traffic as you took the Motorway

late = enumeration_ask('Late', {'Rain': True, 'Traffic': True, 'Motorway': True}, traffic)
late[True]

0.8

### Variable Elimination

In [13]:
q = make_factor('Motorway', {'Rain': False}, traffic)
print(q.variables)
q.cpt

['Motorway']


{(True,): 0.01, (False,): 0.99}

In [14]:
q = make_factor('BossCalls', {'Late': True}, traffic)

In [15]:
q.variables

['BossCalls']

In [16]:
q.cpt

{(True,): 0.8, (False,): 0.19999999999999996}

In [17]:
q = make_factor('Late', {'Rain': True, 'Traffic': True, 'Motorway': True}, traffic)
q.cpt

{(True,): 0.8, (False,): 0.19999999999999996}

### Elimination Ask

In [18]:
elimination_ask('Late', dict(Rain=True, Traffic=True, Motorway=True), traffic).show_approx()

'False: 0.2, True: 0.8'

### Runtime analysis

In [19]:
%%timeit
enumeration_ask('Late', {'Rain': True, 'Traffic': True, 'Motorway': True}, traffic).show_approx()

The slowest run took 4.45 times longer than the fastest. This could mean that an intermediate result is being cached.
136 µs ± 67.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [20]:
%%timeit
make_factor('Late', {'Rain': True, 'Traffic': True, 'Motorway': True}, traffic)

9.71 µs ± 719 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [21]:
%%timeit
elimination_ask('Late', dict(Rain=True, Traffic=True, Motorway=True), traffic).show_approx()

105 µs ± 20.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Approximate Inference in Bayesian Networks

In [22]:
%%timeit
rejection_sampling('Late', dict(Rain=True, Traffic=True, Motorway=True), traffic, 10000)

100 ms ± 8.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%%timeit
likelihood_weighting('Late', dict(Rain=True, Traffic=True, Motorway=True), traffic, 200)

1.88 ms ± 309 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
%%timeit
gibbs_ask('Late', dict(Rain=True, Traffic=True, Motorway=True), traffic, 200)

8.39 ms ± 2.45 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
